In [3]:
#Importing packages/functions
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import random
from random import randrange
#for me to be able to import Jason's module, it seems that I have to have my own copy in the current directory (folder)
from rescurecopyforAdam import initial_random_matrix

In [4]:
#kay so, the problem that came up for me in my intial attempt was that the intial state also plays a role in the event rates
#so if we start by generating our initial state:
t = 0

ratio = (1, 2, 1)

initial_state = initial_random_matrix(3, ratio)

print(initial_state)

[[0. 2. 1.]
 [0. 1. 1.]
 [1. 1. 1.]]


In [5]:
#then we can talk about the KMC rates
#first - the reaction rates for phenol-phenol bonding (pp) and phenol-coal bonding (pc)
#we're assuming they're even, yeah? Because the ring position is what makes the difference?
kpp= 0.8
kpc= 0.8
#I hope you get a kick out of "kpp," Jason :p

#now for the event rates (I'm using arbitrary formulas here)
#KMC_rate_pp = kpp - (kpc*0.2)

#KMC_rate_pc = kpc - (kpp*0.2)

#KMC_rate_no_rxn = 0.4 - (kpc*0.2) - (kpp*0.2)

#But these are incomplete, yeah? We need to start defining our matrix positions

NW = initial_state[0,0]
N = initial_state[0,1]
NE = initial_state[0,2]
W = initial_state[1,0]
#POI = Point of interest, yeah? We can't say MOI for molecule of interest because it could be a void
#The 0 means it's the inital point of interest (because it could change by bonding or translocation)
POI0 = initial_state[1,1]
E = initial_state[1,2]
SW = initial_state[2,0]
S = initial_state[2,1]
SE = initial_state[2,2]

print(NW)
print(N)
print(NE)
print(W)
print(POI0)
print(E)
print(SW)
print(S)
print(SE)


0.0
2.0
1.0
0.0
1.0
1.0
1.0
1.0
1.0


In [6]:
#Cool, so now we gotta re-evaluate our KMC rates, but first we need some sort of modifier based on the quantity of phenols/coals/voids
#surrounding the POI. My first instinct is  we can use the input ratio, but we're talking about the surrounding 8 positions, not all 9
#so I think that's not the move. So let's make sure Adam knows how to index this lovely array that Jason made for us.

p_count = (initial_state == 1.0).sum()
c_count = (initial_state == 2.0).sum()
v_count = (initial_state == 0.0).sum()

print(p_count)
print(c_count)
print(v_count)

6
1
2


In [7]:
#Great! Now, can we exclude the center?
print(initial_state[0,])
p_count_north = (initial_state[0,] ==1.0).sum()
print(p_count_north)

print(initial_state[2,])
p_count_south = (initial_state[2,] ==1.0).sum()
print(p_count_south)

p_count_W = (initial_state[1,0] ==1.0).sum()
p_count_E = (initial_state[1,2] ==1.0).sum()
print(p_count_W)
print(p_count_E)

p_count_surr = (p_count_north + p_count_south + p_count_W + p_count_E)
print(p_count_surr)

[0. 2. 1.]
1
[1. 1. 1.]
3
0
1
5


In [8]:
#now let's get the other objects
c_count_north = (initial_state[0,] ==2.0).sum()
print("Coals in the first row:", c_count_north)

c_count_south = (initial_state[2,] ==2.0).sum()
print("Coals in the third row", c_count_south)

c_count_W = (initial_state[1,0] ==2.0).sum()
c_count_E = (initial_state[1,2] ==2.0).sum()
print("Left coal?", c_count_W)
print("Right coal?", c_count_E)

c_count_surr = (c_count_north + c_count_south + c_count_W + c_count_E)
print("number of surrounding coal molecules:", c_count_surr)

v_count_north = (initial_state[0,] ==0.0).sum()
print("Voids in the first row:", v_count_north)

v_count_south = (initial_state[2,] ==0.0).sum()
print("Voids in the third row", v_count_south)

v_count_W = (initial_state[1,0] ==0.0).sum()
v_count_E = (initial_state[1,2] ==0.0).sum()
print("Left void?", v_count_W)
print("Right void?", v_count_E)

v_count_surr = (v_count_north + v_count_south + v_count_W + v_count_E)
print("number of surrounding voids:", v_count_surr)

Coals in the first row: 1
Coals in the third row 0
Left coal? 0
Right coal? 0
number of surrounding coal molecules: 1
Voids in the first row: 1
Voids in the third row 0
Left void? 1
Right void? 0
number of surrounding voids: 2


In [9]:
#HOORAY THAT WORKED kay so now we gotta make those KMC rates. Maybe we do something like this?
p_coefficient = (p_count_surr/8)
print(p_coefficient)
c_coefficient = (c_count_surr/8)
print(c_coefficient)
v_coefficient = (v_count_surr/8)
print(v_coefficient)

0.625
0.125
0.25


In [10]:
#And then...

#KMC_rate_pp = (kpp - (kpc*0.2))*p_coefficient

#KMC_rate_pc = (kpc - (kpp*0.2))*c_coefficient

#KMC_rate_no_rxn = (0.4 - (kpc*0.2) - (kpp*0.2))*v_coefficient


In [11]:
#So what is the problem with these rates? 
#For one, we are not accounting for the possibility that two phenols next to each other will just not bond (or maybe we are, because that's
#what the 0.4 in the last equation is?)
#same with a phenol and a coal
#we are also assuming that pp and pc bonds are equally likely which I guess is actually true because we're more worried about ring position
#regardless, we gotta do this to make our lives easier:
if POI0 == 0.0:
    KMC_rate_pp = 0

    KMC_rate_pc = 0

    KMC_rate_no_rxn = 1
if POI0 == 1.0:
    KMC_rate_pp = (kpp - (kpc*0.2))*p_coefficient

    KMC_rate_pc = (kpc - (kpp*0.2))*c_coefficient

    KMC_rate_no_rxn = (0.4 - (kpc*0.2) - (kpp*0.2))*v_coefficient
if POI0 == 2.0:
    KMC_rate_pp = 0
    #notice I'm using th p coefficient here, NOT the c coefficient
    KMC_rate_pc = (kpc - (kpp*0.2))*p_coefficient

    KMC_rate_no_rxn = (0.4 - (kpc*0.2) - (kpp*0.2))*v_coefficient

print(KMC_rate_pp)
print(KMC_rate_pc)
print(KMC_rate_no_rxn)

0.4
0.08
0.01999999999999999


In [12]:
#Cool, now for the random choice of the KMC:
bondsum = sum([KMC_rate_pp, KMC_rate_pc])
ratesum = sum([bondsum, KMC_rate_no_rxn])
print(bondsum)
print(ratesum)

event_selector = np.random.rand()
print(event_selector)

chosen_event = ratesum * event_selector
print(chosen_event)

0.48000000000000004
0.5
0.26292208636311565
0.13146104318155782


In [13]:
#and finally:
if 0.0<=chosen_event<=KMC_rate_pp:
        POI = 3.0
        print ('Phenol-Phenol Bond')
if KMC_rate_pp<chosen_event<=bondsum:
        POI = 4.0
        print('Phenol-Coal Bond')
if bondsum<chosen_event<=ratesum:
        POI = POI0
        print('No Reaction')

Phenol-Phenol Bond


In [14]:
#YUH

t_next = t + 1

next_state = initial_state
next_state[1,1]=POI

POI0=POI

print(next_state)

[[0. 2. 1.]
 [0. 3. 1.]
 [1. 1. 1.]]


In [ ]:
#Success! I hope this helps us get closer to a nice working model :)
#Maybe there are parts of this that can become it's own function?
#Note that 3.0 = a phenol bound to a phenol
#BUT! The phenol that the POI bound to did not also change to a 3.0 so there's still more to do